<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_12_28_Configure_LLM_fallbacks_between_OpenAI%2C_Groq%2C_and_Anthropic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to configure LLM fallbacks between models.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [13]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [14]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [15]:
from typing import List, Optional

async def ingest_uri(uri: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_cohere_specification(model: enums.CohereModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Cohere [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.COHERE,
        cohere=input_types.CohereModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_google_specification(model: enums.GoogleModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Google [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.GOOGLE,
        google=input_types.GoogleModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_groq_specification(model: enums.GroqModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Groq [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.GROQ,
        groq=input_types.GroqModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_anthropic_specification(model: enums.AnthropicModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Anthropic [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_deepseek_specification(model: enums.DeepseekModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Deepseek [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.DEEPSEEK,
        deepseek=input_types.DeepseekModelPropertiesInput(
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str, fallbacks: Optional[List[str]] = None):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        ),
        fallbacks=[
            input_types.EntityReferenceInput(
                id=fallback_id
            ) for fallback_id in fallbacks
        ] if fallbacks is not None else None
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_conversation(conversation_id: str):
    if graphlit.client is None:
        return;

    if conversation_id is not None:
        _ = await graphlit.client.delete_conversation(conversation_id)

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return None, None

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id, include_details=True)

        message = response.prompt_conversation.message if response.prompt_conversation is not None else None
        details = response.prompt_conversation.details if response.prompt_conversation is not None else None

        return message, details
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None, None

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [16]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing contents, conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()
await delete_all_contents()

print('Deleted all contents, conversations and specifications.')

content_id = await ingest_uri(uri="https://www.graphlit.com")

if content_id is not None:
    print(f'Ingested content [{content_id}]:')

Deleted all contents, conversations and specifications.
Ingested content [68723ba1-abc7-412a-b8ba-42a19072c23e]:


In [17]:
    # Specify the RAG prompt
    prompt = "In 3-5 detailed paragraphs, explain unstructured data and its usefulness for knowledge capture and retrieval."

Create OpenAI GPT-4o specification, and fallback specifications.

In [18]:
    fallbacks = []
    fallbacks.append(await create_groq_specification(enums.GroqModels.LLAMA_3_3_70B))
    fallbacks.append(await create_anthropic_specification(enums.AnthropicModels.CLAUDE_3_5_SONNET_20241022))
    fallbacks.append(await create_deepseek_specification(enums.DeepseekModels.CHAT))

    specification_id = await create_openai_specification(enums.OpenAIModels.GPT35_TURBO_0613)

    if specification_id is not None:
        # GPT-3.5 Turbo only has 4K token limit, and prompt will fail and fallback to Groq
        print(f'Created OpenAI GPT 3.5 Turbo specification [{specification_id}].')

        print(f'Using [{len(fallbacks)}] fallback specifications.')

        conversation_id = await create_conversation(specification_id, fallbacks)

        if conversation_id is not None:
            print(f'Created conversation [{conversation_id}].')

            message, details = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown(f'### Completion took [{message.completion_time}]:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**'))
                print(message.message)
                print()

            if details is not None:
                display(Markdown('### Details:'))
                display(Markdown(f'**Model**: {details.model_service} {details.model}'))
                display(Markdown(f'**Token Limit**: {details.token_limit}'))
                display(Markdown(f'**Completion Token Limit**: {details.completion_token_limit}'))

            await delete_conversation(conversation_id)

Created OpenAI GPT 3.5 Turbo specification [37db651d-d5eb-4739-afcd-aff1a6f02cb1].
Using [3] fallback specifications.
Created conversation [b87a56a6-f25c-4b58-99a6-e1eac29b892f].


### Completion took [PT3.1164471S]:

**User:**
In 3-5 detailed paragraphs, explain unstructured data and its usefulness for knowledge capture and retrieval.

**Assistant:**

Unstructured data refers to information that does not have a predefined format or organization, making it challenging to search, analyze, and retrieve. This type of data can come in various forms, such as documents, emails, social media posts, audio and video files, and images. Despite its lack of structure, unstructured data contains a vast amount of valuable information that can be leveraged to gain insights, make informed decisions, and capture knowledge. The usefulness of unstructured data lies in its ability to provide context, nuance, and depth to information, which can be difficult to achieve with structured data alone.

The process of capturing and retrieving knowledge from unstructured data involves several steps, including data ingestion, extraction, and analysis. Data ingestion involves collecting and processing unstructured data from various sources, such as web scraping, cloud storage, or social media platforms. Once the data is ingested, extraction techniques, such as nat

### Details:

**Model**: GROQ LLaMA 3.3 70b

**Token Limit**: 8192

**Completion Token Limit**: 820